# FIXME
Some files have multiple entries for a single region for a given time stamp
* Currently, I am taking the first entry

Some files are missing entries for a single hour
* Currently, these are being skipped.

In [ ]:
import pandas as pd
import numpy as np
from glob import glob

In [ ]:
### FIXME


def get_files(year, month):
    files = glob(f'./nyiso_{year}{month:02}/{year}*.csv')
    files.sort()
    dfs = []
    for f in files:
        print(f)
        df = nyiso_to_neat_data(f)
        if type(df) == int:
            continue
        else:
            dfs.append(df)
    return dfs

def nyiso_to_neat_data(fname):
    df = pd.read_csv(fname)
    #print(df.head())
    #print(df.tail())
    mapping = {}
    mapping['date_time'] = []
    mapping['time_zone'] = []
    # Get all regions
    # Some data issues lead to regions having multiple entries
    # in first hour
    start_time = -1
    for idx in df.index:
        if start_time == -1:
            start_time = df.loc[idx, 'Time Stamp']
        if df.loc[idx, 'Name'] not in mapping.keys():
            mapping[df.loc[idx, 'Name']] = []
        if df.loc[idx, 'Time Stamp'] != start_time:
            break
    #print(mapping)
    # Populate values
    for idx in df.index:
        if df.loc[idx, 'Time Stamp'] not in mapping['date_time']:
            mapping['date_time'].append(df.loc[idx, 'Time Stamp'])
            mapping['time_zone'].append(df.loc[idx, 'Time Zone'])
        if len(mapping['date_time']) == len(mapping[df.loc[idx, 'Name']]) + 1:
            mapping[df.loc[idx, 'Name']].append(df.loc[idx, 'Integrated Load'])
    tgt_len = 24
    for k, v in mapping.items():
        if len(v) != tgt_len:
            print(f"Length problem with file {fname}")
            print(k, len(v))
            return -1
    df_new = pd.DataFrame(mapping)
    return df_new

def monthly_file(year, month):
    dfs = get_files(year, month)
    master = dfs[0]
    for i in range(1, len(dfs)):
        master = master.append(dfs[i], ignore_index = True)
    #print(len(master.index))
    master.to_csv(f'./nyiso_{year}{month:02}/demand_summary.csv')

for year in range(2002, 2003):#2019):
    for month in range(1, 13):
        print(year,month)
        monthly_file(year, month)
    
